Skboost

Dit is het gemakkelijkste 

In [10]:
!pip install cmake
!pip uninstall -y skboost
!pip install git+https://github.com/tibovanheule/skboost

  Cloning https://github.com/tibovanheule/skboost to c:\users\seven\appdata\local\temp\pip-req-build-elt4eqc9
  Resolved https://github.com/tibovanheule/skboost to commit 14507560afbfc50cb5a6417cab8e4023b3b58941
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for skboost
Failed to build skboost
  Running setup.py install for skboost: started
  Running setup.py install for skboost: finished with status 'error'


  Running command git clone --filter=blob:none --quiet https://github.com/tibovanheule/skboost 'C:\Users\seven\AppData\Local\Temp\pip-req-build-elt4eqc9'
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [64 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\skboost
  copying skboost\gentleboost.py -> build\lib.win-amd64-cpython-310\skboost
  copying skboost\logitboost.py -> build\lib.win-amd64-cpython-310\skboost
  copying skboost\optimizers.py -> build\lib.win-amd64-cpython-310\skboost
  copying skboost\version.py -> build\lib.win-amd64-cpython-310\skboost
  copying skboost\__init__.py -> build\lib.win-amd64-cpython-310\skboost
  creating build\lib.win-amd64-cpython-310\skboost\datasets
  copying skboost\datasets\__init__.py -> build\lib.win-amd64-cpython-310\skboost\datasets
  creating build\l

In [2]:
import skboost
#help(skboost.milboost.softmax)

In [3]:
#from skboost import milboost
#help(milboost.classifier.MILBoostClassifier)

In [4]:
from loader import parse_c45, bag_set
from __future__ import print_function, division
from sklearn.model_selection import StratifiedKFold
from skboost.milboost.classifier import MILBoostClassifier
from skboost.milboost.softmax import *
from score import result
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import time

In [10]:
# Load list of C4.5 Examples

for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast','musk1']:
    example_set = parse_c45(dataset)

    # Get stats to normalize data
    raw_data = np.array(example_set.to_float())
    data_mean = np.average(raw_data, axis=0)
    data_std  = np.std(raw_data, axis=0)
    data_std[np.nonzero(data_std == 0.0)] = 1.0
    def normalizer(ex):
        ex = np.array(ex)
        if dataset == 'musk1':
            return ex[2:]
        return ex[1:]


    # Group examples into bags
    bagset = bag_set(example_set)

    bags = []
    labels = []

    for bag in bagset: 
        for row in bag.to_float(normalizer):

            labels.append(int(((float(row[-1]) * 2) - 1))) # convert to -1 / 1

            bags.append(list(map(float, row[2:-1])))

    bags = np.array(bags,dtype="float")
    labels = np.array(labels,dtype="int")

    for _ in range(0,5):
        fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
        splittt = 1

        for train_index, test_index in fold.split(bags,labels):
            X_train, X_test = bags[train_index], bags[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            epoch_time = time.time()
            model = MILBoostClassifier(softmax=LogSumExponential(5.0))
            model.fit(X_train,y_train)
            start = time.time()
            predicted = model.predict(X_test)
            time_ep = time.time() - start
            result("MILboost LogSumExponential",y_test,predicted,time_ep, splittt,dataset)
            splittt += 1

        splittt = 1
        for train_index, test_index in fold.split(bags,labels):
            X_train, X_test = bags[train_index], bags[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            epoch_time = time.time()
            model = MILBoostClassifier(softmax=GeneralizedMean(5.0))
            model.fit(X_train,y_train)
            start = time.time()
            predicted = model.predict(X_test)
            time_ep = time.time() - start
            result("MILboost GeneralizedMean",y_test,predicted,time_ep, splittt,dataset)
            splittt += 1

        #splittt = 1
        #for train_index, test_index in fold.split(bags,labels):
         #   X_train, X_test = bags[train_index], bags[test_index]
           # y_train, y_test = labels[train_index], labels[test_index]
            #epoch_time = time.time()
            #model = MILBoostClassifier(softmax=ISR())
            #model.fit(X_train,y_train)
            #start = time.time()
            #predicted = model.predict(X_test)
            #time_ep = time.time() - start
            #result("MILboost ISR",y_test,predicted,time_ep, splittt,dataset)
            #splittt += 1


/Users/tibovanheule/Desktop/MachineLearning/score.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(10, 10))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 

/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tibovanheule/Desktop/MachineLearning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0.0, 1.0, num=100)
t = np.vstack([x, 1 - x]).T
true_t = np.max(t, axis=1)

lse_5 = list(map(LogSumExponential(5.0).f, t))
lse_20 = list(map(LogSumExponential(20.0).f, t))
nor = list(map(NoisyOR().f, t))
isr = list(map(ISR().f, t))
gm_5 = list(map(GeneralizedMean(5.0).f, t))
gm_20 = list(map(GeneralizedMean(20.0).f, t))

plt.plot(x, true_t, 'k--', label='True')
plt.plot(x, lse_5, 'b', label='LSE(5)')
plt.plot(x, lse_20, 'y', label='LSE(20)')
plt.plot(x, nor, 'r', label='NOR')
plt.plot(x, isr, 'g', label='ISR')
plt.plot(x, gm_5, 'm', label='GM(5)')
plt.plot(x, gm_20, 'c', label='GM(20)')

plt.legend()
plt.xlabel('v')
plt.ylabel('g_{l}(v_{l})')

plt.show()
plt.savefig("softmax",dpi=300)
